In [1]:
import sys

import joblib

sys.path.append('../')

from Modules.Utils.Imports import *
from Modules.Utils.ModelWrapper import ModelWrapper
from Modules.Models.BuildBINNs import BINNCovasim_DRUMS

import Modules.Loaders.DataFormatter as DF
import datetime

from utils import plot_loss_convergence, get_case_name
import matplotlib
matplotlib.use('Agg')

c:\Users\Teddy\anaconda3\envs\reu_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

Device set to cpu


In [12]:
path = '../Data/covasim_data/drums_data/'
population = 200000
test_prob = 0.1
trace_prob = 0.3
keep_d = True
retrain = False
dynamic=True
chi_type = 'piecewise'
case_name = get_case_name(population, test_prob, trace_prob, keep_d, dynamic=dynamic, chi_type=chi_type)

params = DF.load_covasim_data(path, population, test_prob, trace_prob, keep_d, case_name,plot=True)

In [13]:
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

def to_numpy(x):
    return x.detach().cpu().numpy()

In [14]:
# split into train/val and convert to torch
data = params['data']
data = (data / params['population']).to_numpy()
params.pop('data')
N = len(data)
split = int(0.8*N)
p = np.random.permutation(N)
x_train = to_torch(p[:split][:, None]/(N-1))
y_train = to_torch(data[p[:split]])
x_val = to_torch(p[split:][:, None]/(N-1))
y_val = to_torch(data[p[split:]])

In [15]:
# generate save path
mydir = os.path.join('../models/covasim', datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
os.makedirs(mydir)

tracing_array = params['tracing_array']

In [16]:
# initialize model
binn = BINNCovasim_DRUMS(params, N - 1, tracing_array, keep_d=keep_d, chi_type=chi_type)
binn.to(device)

BINNCovasim_DRUMS(
  (surface_fitter): main_MLP(
    (mlp): BuildMLP(
      (activation): ReLU()
      (output_activation): ReLU()
      (MLP): Sequential(
        (0): Linear(in_features=1, out_features=512, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=512, out_features=256, bias=True)
        (4): ReLU()
        (5): Dropout(p=0.2, inplace=False)
        (6): Linear(in_features=256, out_features=256, bias=True)
        (7): ReLU()
        (8): Dropout(p=0.2, inplace=False)
        (9): Linear(in_features=256, out_features=9, bias=True)
        (10): ReLU()
      )
    )
    (softmax): Softmax(dim=1)
  )
  (eta_func): infect_rate_MLP(
    (mlp): BuildMLP(
      (activation): ReLU()
      (output_activation): Sigmoid()
      (MLP): Sequential(
        (0): Linear(in_features=3, out_features=256, bias=True)
        (1): ReLU()
        (2): Dropout(p=0.2, inplace=False)
        (3): Linear(in_features=256, out_features=1, bias=

In [17]:
# compile
parameters = binn.parameters()
opt = torch.optim.Adam(parameters, lr=1e-3)
os.makedirs(os.path.join(mydir, case_name))
model = ModelWrapper(
    model=binn,
    optimizer=opt,
    loss=binn.loss,
    augmentation=None,
    # scheduler= scheduler,
    save_name=os.path.join(mydir, case_name) )
model.str_name = 'STEAYDQRF'

In [18]:
# save the range information before training
ranges = [binn.yita_lb, binn.yita_ub, binn.beta_lb, binn.beta_ub, binn.tau_lb, binn.tau_ub, binn.mu_lb, binn.mu_ub]
file_name = '_'.join([str(m) for m in ranges])
joblib.dump(None, os.path.join(mydir, file_name)) # model.save_folder
# if retrain
if retrain:
    model.load(model.save_name + '_best_val_model', device=device)
    model.model.train()
    model.save_name += '_retrain'
epochs = int(2000)
batch_size = 128
rel_save_thresh = 0.05

In [19]:
# train jointly
model.fit(
    x=x_train,
    y=y_train,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=None,
    verbose=1,
    validation_data=[x_val, y_val],
    early_stopping=40000,
    rel_save_thresh=rel_save_thresh)

Epoch 153 | Train loss = 2.6118e+00 | Val loss = 1.1325e+00 | Remaining = 3:35:02           

KeyboardInterrupt: 

In [ ]:
# fitting performance on training data
y_train_pred = to_numpy(model.predict(x_train))


# load training errors
total_train_losses = model.train_loss_list
total_val_losses = model.val_loss_list

In [ ]:
plot_loss_convergence(total_train_losses, total_val_losses, rel_save_thresh, model.save_folder)

In [ ]:
print(model.save_name)

u = binn.surface_fitter(x_val)
u_arr = u.detach().numpy()
u_mean = np.mean(u_arr, axis=0)
# print average values for each solution/compartment
print(u_mean)